In [5]:
#====================================================================================================
# kogpt2 를 이용한 text generation finetuning 훈련 예제
# => https://github.com/mcelikkaya/medium_articles/blob/main/gtp2_training.ipynb
# => https://github.com/gyunggyung/KoGPT2-FineTuning/
#
# => text generation 모델이므로  acc 구하는 것은 의미 없음(*따라서 train loss, val loss 만 구함)
#
# [훈련 dataset]
# => input_ids = <s>text</s>
# => labels = input_ids와 동일
# => attiention_mask
#
# [text generation  훈련 과정]
# 
# 1. gpt-2 모델 선언(GPT2LMHeadModel), tokenizer 선언(PreTrainedTokenizerFast)
# 2. '<s>+문장+</s>' 식으로 된 훈련 dataset 생성
# 3. 모델에 input_ids, lables, attention_mask 을 입력하여 훈련 시킴
# 
# 원래 input_ids = 100,200,101,201,300,301 토큰이 입력된다면, labels은 input_ids 좌측으로 shift된 값
# labels = 200,101,201,300,301 식으로 입력이 이루어 저야 하는데..
# GPT2LMHeadModel 를 이용하면, labels = input_ids 와 똑같이 입력하면됨.
# 이유는 GPT2LMHeadModel 내부적으로, 아래처럼 labels에 대해 shift 연산을 가지고 처리함
#
# [GPT2LMHeadModel 소스 일부]
#if labels is not None:
#        # Shift so that tokens < n predict n
#        shift_logits = lm_logits[..., :-1, :].contiguous()
#        shift_labels = labels[..., 1:].contiguous()
#        # Flatten the tokens
#        loss_fct = CrossEntropyLoss()
#        loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
#====================================================================================================

import torch
from torch.utils.data import Dataset, random_split, DataLoader, RandomSampler, SequentialSampler 
import numpy as np
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast
from transformers import AdamW, get_linear_schedule_with_warmup

from tqdm.notebook import tqdm
import os
import time
from myutils import GPU_info, seed_everything, mlogging, SaveBERTModel, TextGeneration_Dataset


model_path='../model/gpt-2/kogpt-2/'
#model_path='skt/kogpt2-base-v2'

# 출력
OUTPATH = '../model/gpt-2/kogpt-2-ft-0504/'

device = GPU_info()
print(device)

#seed 설정
seed_everything(222)

#logging 설정
logger =  mlogging(loggername="gpt2-ft", logfilename="../log/gpt2-ft")

True
device: cuda:0
cuda index: 0
gpu 개수: 1
graphic name: NVIDIA A30
cuda:0
logfilepath:../log/gpt2-ft_2022-05-11.log


In [6]:

tokenizer = PreTrainedTokenizerFast.from_pretrained(model_path,
                                                   bos_token='</s>',
                                                   eos_token='</s>',
                                                   unk_token='<unk>',
                                                   pad_token='<pad>',
                                                   mask_token='<mask>')


In [ ]:
# 모델 정의 하고, embedding size를 tokenizer 사이즈만큼 조정
model = GPT2LMHeadModel.from_pretrained(model_path)

model.resize_token_embeddings(len(tokenizer))
model.to(device)

In [ ]:
# text generation 테스트 해보는 함수 
def eval_keywords(keywords):
    model.eval()
    
    for keyword in keywords:
        input_seq = "<s>" + keyword
        generated = torch.tensor(tokenizer.encode(input_seq)).unsqueeze(0)
        generated = generated.to(device)
        sample_outputs = model.generate(generated,
                                        do_sample = True,
                                        top_k=30,
                                        max_length=50,
                                        top_p=0.90,
                                        num_return_sequences=2)
        
        for i, sample_output in enumerate(sample_outputs):
            print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))
            if i == 1:
                print("\n")
                                            
        

In [ ]:
# 각 단어를 입력하여, text generation 해 봄
keywords = ["지미 카터","제임스 얼","수학"]
eval_keywords(keywords)

In [7]:
# Fine-Tuning 할 말뭉치를 불러옴 
corpus_path = "../korpora/mycorpus/2_대화체.txt"
all_sentences = []

with open(corpus_path, 'r', encoding='utf-8') as f:
      for line in f:
            all_sentences.append(line.strip())  # strip() 계행문자 제거
   
print(all_sentences[10:15])
print(len(all_sentences))

['주문은 가능하지만  수령은 2달 정도 걸릴 것 같네요', '이런  저희는 물건을 최대한 빠르게 받고 싶어요', '사무용품이나 가구 중 가장 먼저 바꿔야 할 것은 뭐라고 생각하나요', '아무래도 컴퓨터가 아닐까요  가장 많이 쓰니까요', '저는 모두가 같이 쓰는 회의실 책상을 먼저 바꿔야 한다고 생각해요']
100000


In [8]:
# 최대 토큰 계수를 구함.
max_token_len = max([len(tokenizer.encode(s)) for s in tqdm(all_sentences)])
print(f'max_token_len:{max_token_len}')

max_token_len:68


In [9]:
# TextGenerattion 데이터셋 생성
# => bos_token + 문장 + eos_token
dataset = TextGeneration_Dataset(all_sentences, tokenizer, max_length=max_token_len)
print(dataset[10:13])

  0%|          | 0/100000 [00:00<?, ?it/s]

([tensor([    1,  9046, 11031,  9826, 11686,   739, 14525,  8135,  9045,  7187,
         9465, 47203,  9031,  9144,  7098,  8084,     1,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3]), tensor([    1, 10165,   739,  9265, 22633, 14579, 19226, 15292, 10884, 25799,
         8084,     1,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,    

In [ ]:
# 훈련 데이터와 test 데이터를 나눔
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

train_set, val_set = random_split(dataset, [train_size, val_size])

print(f'train_size:{len(train_set)}')
print(f'val_size:{len(val_set)}')


In [ ]:
# dataloader 생성
batch_size = 32
train_loader = DataLoader(train_set, sampler = RandomSampler(train_set), batch_size=batch_size)
eval_loader = DataLoader(val_set, sampler = RandomSampler(val_set), batch_size=batch_size)

In [ ]:
for val_data in eval_loader:
    print(val_data)
    break

In [ ]:
# 훈련 시작 

##################################################
epochs = 2            # epochs
learning_rate = 3e-5  # 학습률
##################################################

# optimizer 적용
optimizer = AdamW(model.parameters(), 
                 lr=learning_rate, 
                 eps=1e-8) # 0으로 나누는 것을 방지하기 위한 epsilon 값(10^-6 ~ 10^-8 사이 이값 입력합)

# 총 훈련과정에서 반복할 스탭
total_steps = len(train_loader)*epochs
warmup_steps = total_steps * 0.1 #10% of train data for warm-up

# 손실률 보여줄 step 수
p_itr = int(len(train_loader)*0.1)  
    
# step마다 모델 저장
save_steps = int(total_steps * 0.5)
    
# 스캐줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=warmup_steps, 
                                            num_training_steps=total_steps)

itr = 1

total_loss = 0
total_len = 0
total_test_len = 0
total_correct = 0
total_test_loss = 0
total_test_loss_count = 0
total_test_correct = 0

list_train_loss = []
list_validation_loss = []

# 그래디언트 초기화(*set_to_none=True 로 설정하면, 그래디언트 업데이트시, 쓰기작업만 수행되어 속도가 빨라진다)
model.zero_grad(set_to_none=True)
for epoch in tqdm(range(epochs)):

    model.train() # 훈련모드로 변환
    for data in tqdm(train_loader):
        model.zero_grad(set_to_none=True)# 그래디언트 초기화(*set_to_none=True 로 설정하면, 그래디언트 업데이트시, 쓰기작업만 수행되어 속도가 빨라진다)
        
        # 입력 값 설정
        input_ids = data[0].to(device)
        attention_mask = data[1].to(device)
        
         # labels은 input_ids와 동일하게 입력 (*GPT2LMHeadModel 을 이용하는 경우, 내부적으로 labels 값에 대해 shift 연산처리를 해서 손실 구함)
        labels = data[0].to(device)  
        #print('Labels:{}'.format(labels))
        
        # 모델 실행
        outputs = model(input_ids=input_ids, 
                        attention_mask=attention_mask,
                        labels=labels)
        
       
        # 출력값 loss,logits를 outputs에서 얻어옴
        loss = outputs.loss
        logits = outputs.logits
        #print('Loss:{}, logits:{}'.format(loss, logits))
        
        # logits_shape: torch.Size([32, 68, 51200])
        # => batch_size, sequence_max_len, token_len
        #print(f'logits_shape: {logits.shape}')                    
        
        # optimizer 과 scheduler 업데이트 시킴
        loss.backward()   # backward 구함
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)   # 그래디언트 클리핑 (gradient vanishing이나 gradient exploding 방지하기 위한 기법)
        optimizer.step()  # 가중치 파라미터 업데이트(optimizer 이동)
        scheduler.step()  # 학습률 감소
        
        # ***further pretrain 에는 손실률 계산을 넣지 않음
        # 정확도 계산하는 부분은 no_grade 시켜서, 계산량을 줄임.
        
        # => torch.no_grad()는 gradient을 계산하는 autograd engine를 비활성화 하여 
        # 필요한 메모리를 줄이고, 연산속도를 증가시키는 역활을 함
        with torch.no_grad():
            
            # 손실률 계산
            total_loss += loss.item()
                
            #===========================================
            # 정확도(Accurarcy) 계산
            #correct = AccuracyForMLM(logits, labels, attention_mask)           
            #total_correct += correct.sum().item() 
            #total_len += attention_mask.sum().item()
            #=========================================
     
            # 주기마다 test(validataion) 데이터로 평가하여 손실류 계산함.
            if itr % p_itr == 0:
                
                train_loss = total_loss/p_itr
                #train_acc = total_correct/total_len
                
                ####################################################################
                # 주기마다 eval(validataion) 데이터로 평가하여 손실류 계산함.
                # 평가 시작
                model.eval()
                for data in tqdm(eval_loader):
                #for data in eval_loader:
                    # 입력 값 설정
                    input_ids = data[0].to(device)
                    attention_mask = data[1].to(device)
                    labels = data[0].to(device)
                    
                    with torch.no_grad():
                        # 모델 실행
                        outputs = model(input_ids=input_ids, 
                                       attention_mask=attention_mask,
                                       labels=labels)

                        # 출력값 loss,logits를 outputs에서 얻어옴
                        loss = outputs.loss
                        logits = outputs.logits
                        
                        total_test_loss += loss.item()
                        total_test_loss_count += 1
                        
                        #===========================================
                        # 정확도(Accurarcy) 계산
                        #correct = AccuracyForMLM(logits, labels, attention_mask)           
                        #total_test_correct += correct.sum().item() 
                        #total_test_len += attention_mask.sum().item()
                        #=========================================
                        
                #val_acc = total_test_correct/total_test_len        
                val_loss = total_test_loss/total_test_loss_count
                    
                logger.info('[Epoch {}/{}] Iteration {} -> Train Loss: {:.4f}, Val Loss: {}'.format(epoch+1, epochs, itr, train_loss, val_loss))
                #logger.info('[Epoch {}/{}] Iteration {} -> Train Loss: {:.4f}, Train Acc: {:.4f}, Val Loss: {}, Val Acc:{}({}/{})'.format(epoch+1, epochs, itr, train_loss, train_acc, val_loss, val_acc, total_test_correct, total_test_len))
                    
                list_train_loss.append(train_loss)
                list_validation_loss.append(val_loss)
                 
                # 변수들 초기화    
                total_loss = 0
                total_test_loss = 0
                total_test_correct = 0
                total_test_len = 0
                total_correct = 0
                total_test_loss = 0
                total_test_loss_count = 0
                ####################################################################
            if itr % save_steps == 0:
                #전체모델 저장
                SaveBERTModel(model, tokenizer, OUTPATH, epochs, learning_rate, batch_size)

        itr+=1

In [ ]:
# 모델 저장
SaveBERTModel(model, tokenizer, OUTPATH, epochs, learning_rate, batch_size)

In [ ]:
# 그래프로 loss 표기
#!pip install matplotlib
import matplotlib.pyplot as plt

plt.plot(list_train_loss, label='Train Loss')
plt.plot(list_validation_loss, label='val Loss')
#plt.plot(list_validation_acc, label='Eval Accuracy')
plt.legend()
plt.show()

